In [10]:
%load_ext bigdata
%pig_start
%timeout 300
!hadoop fs -rm data.tsv .
!hadoop fs -put data.tsv .
!hadoop fs -ls

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata
rm: `data.tsv': No such file or directory
rm: `.': Is a directory
Found 2 items
-rw-r--r--   1 root supergroup       1780 2020-02-16 16:12 data.tsv
-rw-r--r--   1 root supergroup    2271958 2020-02-16 15:48 truck_event_text_partition.csv


In [11]:
%%pig
records = LOAD 'data.tsv' AS (c1:CHARARRAY, c2:CHARARRAY, c3:CHARARRAY);
DUMP records

 records = LOAD 'data.tsv' AS (c1:CHARARRAY, c2:CHARARRAY, c3:CHARARRAY);
 DUMP records
2020-02-16 16:12:53,599 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:12:53,898 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2020-02-16 16:12:53,908 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2020-02-16 16:12:53,924 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.replication
2020-02-16 16:12:54,526 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker.http.address is deprecated. Instead, use mapreduce.jobtracker.http.address
2020-02-16 16:12:54,538 [JobControl] INF

In [28]:
%%pig
second_col = FOREACH records GENERATE c2;
second_col_b = FOREACH second_col GENERATE REPLACE(c2, '\\}', '') AS c2;
second_col_c = FOREACH second_col_b GENERATE REPLACE(c2, '\\)', '') AS c2;
second_col_d = FOREACH second_col_c GENERATE REPLACE(c2, '\\{', '') AS c2;
second_col_e = FOREACH second_col_d GENERATE REPLACE(c2, '\\(', '') AS c2;
DUMP second_col_e;

 second_col = FOREACH records GENERATE c2;
 second_col_b = FOREACH second_col GENERATE REPLACE(c2, '\\}', '') AS c2;
;second_col_c = FOREACH second_col_b GENERATE REPLACE(c2, '\\)', '') AS c2 
;second_col_d = FOREACH second_col_c GENERATE REPLACE(c2, '\\{', '') AS c2 
;second_col_e = FOREACH second_col_d GENERATE REPLACE(c2, '\\(', '') AS c2 
 DUMP second_col_e;
2020-02-16 16:40:53,842 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:40:54,773 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:40:54,787 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2020-02-16 16:40:54,797 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2020-02-16 16:40:54,838 [JobControl] INFO  org.apache.hadoop.mapreduce.J

In [35]:
%%pig
div = FOREACH second_col_e GENERATE FLATTEN(TOKENIZE(c2)) AS c2;
grouped = GROUP div BY c2;
letcount = FOREACH grouped GENERATE group, COUNT(div);
DUMP letcount;

 div = FOREACH second_col_e GENERATE FLATTEN(TOKENIZE(c2)) AS c2;
 grouped = GROUP div BY c2;
 letcount = FOREACH grouped GENERATE group, COUNT(div);
 DUMP letcount;
2020-02-16 16:48:19,917 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:48:20,680 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:48:20,712 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2020-02-16 16:48:20,726 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2020-02-16 16:48:21,176 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2020-02-16 16:48:21,218 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1581861269524_0062
2020-02-16 16:48:21,223 [JobCo

In [36]:
%%pig
STORE letcount INTO 'output';

 STORE letcount INTO 'output';
2020-02-16 16:52:05,340 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.textoutputformat.separator is deprecated. Instead, use mapreduce.output.textoutputformat.separator
2020-02-16 16:52:05,454 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:52:05,632 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-02-16 16:52:05,659 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2020-02-16 16:52:05,680 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2020-02-16 16:52:06,131 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2020-02-16 16:52:06,191 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for

In [37]:
%%pig
fs -get output/ .

 fs -get output/ .


In [38]:
!hadoop fs -rm output/*
!hadoop fs -rmdir output

Deleted output/_SUCCESS
Deleted output/part-r-00000


In [39]:
%pig_quit